### 출처

https://various.foodsafetykorea.go.kr/nutrient/general/down/historyList.do

- 한글: 식품영양성분 데이터베이스
- 영문: Korean Food Composition Database system(K-FCDB)

모든 식품의 영양성분함량기준량은 100g임.

In [3]:
!pip install mysql-connector-python

In [2]:
!pip install SQLAlchemy

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [3]:
# 기본적인 읽기
df = pd.read_excel('./20240807_음식DB.xlsx')
df

,식품코드,식품명,데이터구분코드,데이터구분명,식품기원코드,식품기원명,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,트랜스 리놀렌산(18:3t)(mg),카페인(mg),출처코드,출처명,식품중량,업체명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자
0,D101-004160000-0001,국밥_돼지머리,D,음식,1,가정식(분석 함량),1,밥류,1004,국밥,...,0.0,-,3,식품의약품안전처,900g,해당없음,1,분석,2017-12-31,2024-08-02
1,D101-004310000-0001,국밥_순대국밥,D,음식,1,가정식(분석 함량),1,밥류,1004,국밥,...,0.0,-,3,식품의약품안전처,900g,해당없음,1,분석,2017-12-31,2024-08-02
2,D101-004500000-0001,국밥_콩나물,D,음식,1,가정식(분석 함량),1,밥류,1004,국밥,...,0.0,-,3,식품의약품안전처,780g,해당없음,1,분석,2018-12-31,2024-08-02
3,D101-006000000-0001,기장밥,D,음식,1,가정식(분석 함량),1,밥류,1006,기장밥,...,0.0,-,3,식품의약품안전처,200g,해당없음,1,분석,2017-12-31,2024-08-02
4,D101-007000000-0001,김밥,D,음식,1,가정식(분석 함량),1,밥류,1007,김밥,...,0.0,-,3,식품의약품안전처,230g,해당없음,1,분석,2018-12-31,2024-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12146,D714-650000000-0001,청포묵무침,D,음식,7,산업체급식(재료량 기반 산출 함량),14,생채·무침류,14650,청포묵무침,...,NaN,NaN,3,식품의약품안전처,150ml,해당없음,3,산출,2022-11-30,2024-08-02
12147,D714-651000000-0001,치커리겉절이,D,음식,7,산업체급식(재료량 기반 산출 함량),14,생채·무침류,14651,치커리겉절이,...,NaN,NaN,3,식품의약품안전처,180ml,해당없음,3,산출,2022-11-30,2024-08-02
12148,D714-655362100-0001,파래무침_파래_무,D,음식,7,산업체급식(재료량 기반 산출 함량),14,생채·무침류,14655,파래무침,...,NaN,NaN,3,식품의약품안전처,130ml,해당없음,3,산출,2022-11-30,2024-08-02
12149,D718-706000000-0001,짜장소스,D,음식,7,산업체급식(재료량 기반 산출 함량),18,"장류, 양념류",18706,짜장소스,...,NaN,NaN,3,식품의약품안전처,200ml,해당없음,3,산출,2022-11-30,2024-08-02


영양성분함량기준량 = 100g

In [8]:
print(len(df.columns))  # 전체 열 개수

108


In [7]:
pd.set_option('display.max_columns', None)
print(df.columns)

Index(['식품코드', '식품명', '데이터구분코드', '데이터구분명', '식품기원코드', '식품기원명', '식품대분류코드',
       '식품대분류명', '대표식품코드', '대표식품명',
       ...
       '트랜스 리놀렌산(18:3t)(mg)', '카페인(mg)', '출처코드', '출처명', '식품중량', '업체명',
       '데이터생성방법코드', '데이터생성방법명', '데이터생성일자', '데이터기준일자'],
      dtype='object', length=108)


In [18]:
df_update = df[['식품명', '식품대분류명', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '당류(g)', '식품중량']]
df_update = df_update.dropna() # 결측치 제거
df_update.reset_index(drop=True, inplace=True)
df_update

,식품명,식품대분류명,에너지(kcal),탄수화물(g),단백질(g),지방(g),당류(g),식품중량
0,국밥_돼지머리,밥류,137,15.94,6.70,5.16,0.16,900g
1,국밥_순대국밥,밥류,75,10.38,3.17,2.28,0.17,900g
2,국밥_콩나물,밥류,52,10.93,1.45,0.24,0.00,780g
3,기장밥,밥류,166,36.77,3.44,0.57,0.00,200g
4,김밥,밥류,140,19.98,4.84,4.55,0.00,230g
...,...,...,...,...,...,...,...,...
3143,청포묵무침,생채·무침류,43,6.54,1.30,1.68,0.72,150ml
3144,치커리겉절이,생채·무침류,22,3.51,0.72,0.73,1.89,180ml
3145,파래무침_파래_무,생채·무침류,23,4.86,1.07,0.38,2.62,130ml
3146,짜장소스,"장류, 양념류",100,12.86,4.38,3.64,4.16,200ml


In [19]:
# food_data.csv로 저장
df_update.to_csv('food_data.csv',
              index=False,              # 인덱스 제외
              encoding='utf-8-sig',     # 한글 깨짐 방지
              sep=',',                  # 구분자 지정
              na_rep='NaN')            # 결측값 표현 방식 지정

In [4]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'diet_admin',
    'password': 'pproject2024!',
    'database': 'diet_recommendation_db'
}

# CSV 파일 읽기
df = pd.read_csv('food_data.csv')

# 열 이름 변경
df_renamed = df.rename(columns={
    '식품명': 'name',
    '식품대분류명': 'category',
    '에너지(kcal)': 'calories',
    '탄수화물(g)': 'carbs',
    '단백질(g)': 'protein',
    '지방(g)': 'fat',
    '당류(g)': 'sugar',
    '식품중량': 'serving_size'
})

# MySQL 연결 및 데이터 삽입
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")
df_renamed.to_sql('foods', con=engine, if_exists='append', index=False)

3148